In [ ]:
library(tidyverse)

In [2]:
zip_file = "../input/EPA_DATA.zip"
if (!file.exists(zip_file)) {
    url <- "https://d396qusza40orc.cloudfrontnet/exdata%2Fdata%2FNEI_data.zip"
    download.file(url, destfile = zip_file)
    unzip(zip_file, exdir = "../input")
}

Warning message in download.file(url, destfile = zip_file):
"URL 'https://d396qusza40orc.cloudfrontnet/exdata%2Fdata%2FNEI_data.zip': lo status era 'Couldn't resolve host name'"


ERROR: Error in download.file(url, destfile = zip_file): non `e possibile aprire URL 'https://d396qusza40orc.cloudfrontnet/exdata%2Fdata%2FNEI_data.zip'


In [ ]:
NEI <- readRDS("../input/summarySCC_PM25.rds")
SCC <- readRDS("../input/Source_Classification_Code.rds")

NEI$type <- as.factor(NEI$type)
NEI$year <- as.factor(NEI$year)

In [ ]:
# 1st assignment from: Instructions.md#Assignment
# 1.Have total emissions from PM2.5 decreased in the United States from 1999 to 2008?
# Using the base plotting system, make a plot showing the total PM2.5 emission from all sources
# for each of the years 1999, 2002, 2005, and 2008.

if (!"NEI" %in% ls()) {
    source("readData.R")
}

pm <- NEI %>%
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

# saveing plot into an image file
png("plot1.png")
plot(
    x = as.numeric(levels(pm$year)[pm$year]),
    y = pm$TotalEmissions,
    pch = 16, col = "black", cex = 2,
    xlab = "Year", ylab = "Total Emissions",
    main = "Total Emissions from PM2.5",
)

dev.off()

In [ ]:
# 2nd assignment from: Instructions.md#Assignment
# Have total emissions from PM2.5 decreased in the Baltimore City,
# Maryland (fips == "24510") from 1999 to 2008?
# Use the base plotting system to make a plot answering this question.

if (!"NEI" %in% ls()) {
    source("readData.R")
}

pm <- NEI %>%
    subset(fips == "24510") %>% # fips=24510 for Baltimore City
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

png("plot2.png")
plot(
    x = as.numeric(levels(pm$year)[pm$year]),
    y = pm$TotalEmissions,
    pch = 16, col = "black", cex = 2,
    xlab = "Year", ylab = "Total Emissions",
    main = "Total Emissions from PM2.5 in Baltimore City, Maryland",
)
dev.off()

In [ ]:
# 3rd assignment from: Instructions.md#Assignment
# Of the four types of sources indicated by the type
# (point, nonpoint, onroad, nonroad)variable,
# which of these four sources have seen decreases in emissions from 1999–2008 for Baltimore City?
# Which have seen increases in emissions from 1999–2008?
# Use the ggplot2 plotting system to make a plot answer this question.

if (!"NEI" %in% ls()) {
    source("readData.R")
}

pm <- NEI %>%
    subset(fips == "24510") %>% # fips=24510 for Baltimore City
    group_by(year, type) %>%
    summarise(TotalEmissions = sum(Emissions))

ggplot(pm) +
    geom_line(aes(x = year, y = TotalEmissions, color = type, group = type)) +
    geom_point(aes(x = year, y = TotalEmissions, color = type)) +
    labs(x = "Year", y = "Total Emissions", title = "Baltimore City, Maryland")
ggsave("plot3.png")

In [ ]:
# 4th assignment from: Instructions.md#Assignment
# Across the United States, how have emissions from coal combustion-related
# sources changed from 1999–2008?

if (!"NEI" %in% ls()) {
    source("readData.R")
}

# I'm assumming any coal combustion related source has "Coal" in its EI.Sector
coalComb <- dplyr::filter(SCC, grepl("Coal", EI.Sector))

pm <- NEI %>%
    filter(SCC %in% coalComb$SCC) %>%
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

ggplot(pm) +
    geom_line(aes(x = year, y = TotalEmissions, group = 1)) +
    geom_point(aes(x = year, y = TotalEmissions), size = 2.5) +
    labs(x = "Year", y = "Total Emissions", title = "Total Emissions from Coal Combustion-Related Sources")
ggsave("plot4.png")

In [ ]:
# 5th assignment from: Instructions.md#Assignment
# How have emissions from motor vehicle sources changed from 1999–2008 in Baltimore City?

if (!"NEI" %in% ls()) {
    source("readData.R")
}

# I'm assumming all motor vehicle sources have "Motor Vehicle" in their Short.Name
motorVehicle <- dplyr::filter(SCC, grepl("Vehicle", Short.Name))

pm <- NEI %>%
    filter(SCC %in% motorVehicle$SCC) %>%
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

ggplot(pm) +
    geom_line(aes(x = year, y = TotalEmissions, group = 1)) +
    geom_point(aes(x = year, y = TotalEmissions), size = 2.5) +
    labs(x = "Year", y = "Total Emissions", title = "Motor Vehicles Emission")
ggsave("plot5.png")

In [ ]:
# 6th assignment from: Instructions.md#Assignment
# Compare emissions from motor vehicle sources in Baltimore City
# with emissions from motor vehicle sources in Los Angeles County,
# California fips == "06037".
# Which city has seen greater changes over time in motor vehicle emissions?
if (!"NEI" %in% ls()) {
    source("readData.R")
}

motorVehicle <- dplyr::filter(SCC, grepl("Vehicle", Short.Name))

pmLA <- NEI %>%
    filter(SCC %in% motorVehicle$SCC) %>%
    subset(fips == "06037") %>%
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

pmBA <- NEI %>%
    filter(SCC %in% motorVehicle$SCC) %>%
    subset(fips == "24510") %>%
    group_by(year) %>%
    summarise(TotalEmissions = sum(Emissions))

pmBA$city <- rep("Baltimore", 4)
pmLA$city <- rep("Los Angeles", 4)
pm <- rbind(pmBA, pmLA)
pm$city <- as.factor(pm$city)

ggplot(data = pm) +
    geom_point(aes(x = year, y = TotalEmissions, group = city), size = 2.5) +
    geom_line(aes(x = year, y = TotalEmissions, group = city, color = city)) +
    labs(x = "Year", y = "Total Emissions", title = "Baltimore vs. Los Angeles")
ggsave("plot6.png")